In [120]:
import torch
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset, DataLoader
import zuko

In [121]:
mu = 42
N = 50000
prior_over_mu = torch.distributions.Normal(torch.Tensor([0.0]), torch.Tensor([25.0]))

Utils

In [122]:
def true_data_generating_process():
    dist = torch.distributions.Normal(mu, 1.0)
    return dist.sample((100, ))

def computing_summary(data):
    sample_mean = torch.mean(data)
    sample.var = torch.var(data)
    return sample_mean,  sample_var


Simulation (line 1 & 2 of Algorithm 1)

In [123]:
def simulator(mu):
    dist = torch.distributions.Normal(mu, 2.0)
    sample = dist.sample((100, ))
    sample_mean = torch.mean(sample)
    sample_var = torch.var(sample)
    return sample_mean, sample_var


thetas_vect = torch.zeros((N, 1))
X_vect = torch.zeros((N, 2))
for i in range(N):
    theta_i = prior_over_mu.sample((1, ))
    x_i = simulator(theta_i)
    thetas_vect[i] = theta_i
    X_vect[i][0], X_vect[i][1] = x_i

thetas_test = torch.zeros((N//10, 1))
x_test = torch.zeros((N//10, 2))  
for i in range(N//10):
    theta_i_test = prior_over_mu.sample((1, ))
    x_i_test = simulator(theta_i_test)
    thetas_test[i] = theta_i_test
    x_test[i][0], X_vect[i][1] = x_i_test

In [124]:
#flowjax.bijections.transformers import RationalQuadraticSplineTransformer # No jax available ... 

Train NPE q(theta|x) on the simulated dataset {(thetai, xi)}, i=1, ... N
Uses a neural spline flow defining the transform on the interval [-5, 5] using 10  spline segments and 5 coupling layers. The base of the flow is a standard Gaussian distribution. 

In [125]:
#D = TensorDataset(thetas_vect,X_vect) #D=  (thetai, xi)

#q_NPE = RationalQuadraticSplineTransformer(10, 5)
q_NPE = zuko.flows.NSF(1,2, hidden_features=[5]*1) #TODO Essayer de comprendre comment il est crée pck là ...

In [126]:
dataset = TensorDataset(thetas_vect, X_vect)
batch_size = 256
loader = DataLoader(dataset,batch_size,shuffle=True)

In [127]:
x_test.size()

torch.Size([5000, 2])

In [128]:
x_test.size()

torch.Size([5000, 2])

In [129]:
max_epochs = 50
learning_rate_NSF = 5*10e-4
epsilon = 1e-3
optimizer = torch.optim.Adam(q_NPE.parameters(), learning_rate_NSF)
with torch.no_grad():
    previous_loss = -q_NPE(x_test).log_prob(thetas_test).mean()
counter=1
patience = 5
for epoch in range(max_epochs):
    print(f" Epoch = {epoch+1}")
    for batch_idx, batch in enumerate(loader):
        thetas_batch, x_batch = batch
        #print(q_NPE)
        loss = -q_NPE(x_batch).log_prob(thetas_batch).mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        loss_test = -q_NPE(x_test).log_prob(thetas_test).mean()
        if np.abs(loss_test - previous_loss) < epsilon:
            counter += 1
        if counter == patience:
            print(f"Early stop at iteration {i}")
            break
        previous_loss = loss_test
    print(f"Loss on the validation set = {previous_loss}")

 Epoch = 1
Loss on the validation set = 316.3840637207031
 Epoch = 2
Loss on the validation set = 316.1841125488281
 Epoch = 3
Loss on the validation set = 316.193115234375
 Epoch = 4
Loss on the validation set = 316.1728515625
 Epoch = 5
Loss on the validation set = 316.1843566894531
 Epoch = 6
Loss on the validation set = 316.1733093261719
 Epoch = 7
Loss on the validation set = 316.2033386230469
 Epoch = 8
Loss on the validation set = 316.1696472167969
 Epoch = 9
Loss on the validation set = 316.1954040527344
 Epoch = 10
Loss on the validation set = 316.1796569824219
 Epoch = 11
Loss on the validation set = 316.1809387207031
 Epoch = 12
Loss on the validation set = 316.197998046875
 Epoch = 13
Loss on the validation set = 316.19573974609375
 Epoch = 14
Loss on the validation set = 316.1761779785156
 Epoch = 15
Loss on the validation set = 316.1553039550781
 Epoch = 16
Loss on the validation set = 316.176025390625
 Epoch = 17
Loss on the validation set = 316.157958984375
 Epoch = 18


Train q(x) on {xi} i=1, ... N (so first gen the x)
For q(x) uses of a block neural autoregressive flow, single hidden layer of size 8D, x in R^d

In [130]:
x = torch.zeros((N, 2))
for i in range(N):
    theta_tmp = prior_over_mu.sample((1, ))
    xi = simulator(theta_tmp)
    x[i][0], x[i][1] = xi
# {xi} i=1, ... N 
xtest = torch.zeros((N//10, 2))
for i in range(N//10):
    theta_tmp = prior_over_mu.sample((1, ))
    xtest[i][0], xtest[i][1] =  simulator(theta_tmp)


In [131]:
dataset = TensorDataset(x)
batch_size = 256
loader = DataLoader(dataset,batch_size,shuffle=True)


In [132]:
q_x = ??? #Pas trouvé dans le package encore. 
max_epochs = 50
learning_rate_BNAF = 10-2
epsilon = 1e-5
optimizer = torch.optim.Adam(q_x.parameters(), learning_rate_BNAF)
with torch.no_grad():
    previous_loss = -q_x.log_prob(xtest).mean()
counter=1
patience = 5
for epoch in range(max_epochs):
    print(f" Epoch = {epoch+1}")
    for batch_idx, batch in enumerate(loader):
        thetas_batch, x_batch = batch
        loss = -q_x.log_prob(xtest).mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        loss_test = -q_x.log_prob(xtest).mean()
        if np.abs(loss_test - previous_loss) < epsilon:
            counter += 1
        if counter == patience:
            print(f"Early stop at iteration {i}")
            break
        previous_loss = loss_test
    print(f"Loss on the validation set = {previous_loss}")

SyntaxError: invalid syntax (1650208075.py, line 1)

Il faut aussi faire les 2 autres simu cad NPE (comme avant) et en mettant directement le modele d'erreur dans le simu